In [1]:
import numpy as np

# Duals
check out https://www.wikiwand.com/en/Dual_number#/Differentiation

In [36]:
class DualNumber:
    def __init__(self, real, dual):
        self.real = real
        self.dual = dual

    def __toDual(self,other):
        if isinstance(other, DualNumber): return other
        return DualNumber(other,0)
        
        
    def __add__(self, other1):
        other = self.__toDual(other1)
        return DualNumber(self.real + other.real,
                          self.dual + other.dual)
    __radd__ = __add__

    def __sub__(self, other1):
        other = self.__toDual(other1)
        return DualNumber(self.real - other.real,
                          self.dual - other.dual)

    def __rsub__(self, other):
        return DualNumber(other, 0) - self

    def __mul__(self, other1):
        other = self.__toDual(other1)
        return DualNumber(self.real * other.real,
                          self.real * other.dual + self.dual * other.real)

    __rmul__ = __mul__

    def __truediv__(self, other1):
        other = self.__toDual(other1)
        return DualNumber(self.real/other.real,
                          (self.dual*other.real - self.real*other.dual)/(other.real**2))

    def __rtruediv__(self, other1):
        other = self.__toDual(other1)
        return other/self

    def __pow__(self, other):
        return DualNumber(self.real**other,
                          self.dual * other * self.real**(other - 1))

    def conj(self):
        if not isinstance(self.real,bool) and not isinstance(self.real,int) and not isinstance(self.real,float):
            return DualNumber(self.real,-self.dual)
        elif sinstance(self.real,complex):
            return DualNumber(self.real.conjugate(),-self.dual.conjugate())
        return DualNumber(self.real.conjugate,-self.dual.conjugate)
    
    def __repr__(self):
        return repr(self.real) + ' + ' + repr(self.dual) + '*'+'\u03B5'

In [37]:
DualNumber(0,10)

0 + 10*ε

In [38]:
A = DualNumber(1,10)
B = DualNumber(1,5)
A*B

10 + 105*ε

In [4]:
complex("1+j").conjugate

<function complex.conjugate>

In [5]:
DualNumber(0,10)*DualNumber(0,10)

0 + 0*ε

In [6]:
DualNumber(1,10)**5

1 + 50*ε

In [7]:
DualNumber(1,10)/DualNumber(1,10)

1.0 + 0.0*ε

In [8]:
A = DualNumber(1,10)
B = DualNumber(0,10)
A*B
B**2

0 + 0*ε

# Quaternions

install pyquaternion if you havent already, just run the next block as code in the notebook

import sys
!{sys.executable} -m pip install pyquaternion

import numpy as np
np.set_printoptions(suppress=True)
from pyquaternion import Quaternion

In [39]:
q1 = Quaternion(axis=[1, 0, 0], angle=np.pi)
print(q1)

0.000 +1.000i +0.000j +0.000k


In [40]:
q2 = Quaternion(axis=[0, 1, 0], angle=np.pi/ 2) 
print(q2)

0.707 +0.000i +0.707j +0.000k


In [41]:
q3 = q1 * q2
print(q3)

0.000 +0.707i +0.000j +0.707k


In [42]:
q3.conjugate

Quaternion(4.329780281177467e-17, -0.7071067811865476, -4.329780281177467e-17, -0.7071067811865476)

In [43]:
v = np.array([0., 0., 1.])
q3.rotate(v)

array([1., 0., 0.])

In [44]:
q3* Quaternion(0,v[0],v[1],v[2])*q3.conjugate

Quaternion(0.0, 1.0000000000000002, 0.0, 0.0)

In [15]:
q3.rotation_matrix

array([[ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 1.,  0.,  0.]])

In [16]:
q3.transformation_matrix

array([[ 0.,  0.,  1.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.]])

# Dual Quaternions
If you really want to blow your mind check out Dual quaternions and try it
https://www.wikiwand.com/en/Dual_quaternion
https://www.euclideanspace.com/maths/algebra/realNormedAlgebra/other/dualQuaternion/index.htm

https://github.com/Achllle/dual_quaternions

In [45]:
from dual_quaternions import DualQuaternion

In [47]:
P_start = [1,2,3]
Q_start = Quaternion([1]+P_start)
DQ_start = DualQuaternion(Quaternion(),Q_start)

D_translate = [10,10,0]
Dn = np.array([0]+D_translate)/2

TranslateQ =  Quaternion(Dn)
TranslateDQ = DualQuaternion(Quaternion(),TranslateQ)

print(TranslateDQ*DQ_start*TranslateDQ.combined_conjugate())

rotation: Quaternion(1.0, 0.0, 0.0, 0.0), translation: Quaternion(1.0, 11.0, 12.0, 3.0), 
translation vector: [22.0, 24.0, 6.0]


In [34]:
def rotateAroundPoint(point,quat):
    D_translate = point
    Dn = np.array([0]+D_translate)/2
    Dinv = -np.array([0]+D_translate)/2
    
    TDQ =  DualQuaternion(Quaternion(),Quaternion(Dn))
    TDQinv = DualQuaternion(Quaternion(),Quaternion(Dinv))

    RotateDQ = DualQuaternion(quat,Quaternion())
    return TDQ*RotateDQ*TDQinv

def ptFromDQ(dq):
    q = dq.q_d
    return q.vector

In [50]:
point = [10,10,0]
AboutP_DQ = rotateAroundPoint(point,q2)

DQ_end = AboutP_DQ*DQ_start*AboutP_DQ.combined_conjugate()

print(DQ_end)

ptFromDQ(DQ_end)


rotation: Quaternion(1.0000000000000002, 0.0, 0.0, 0.0), translation: Quaternion(1.0, 13.000000000000004, 0.5857864376269052, 9.0), 
translation vector: [26.000000000000014, 1.1715728752538106, 18.000000000000004]


array([13.        ,  0.58578644,  9.        ])